In [1]:
#import required lib functions 
from __future__ import print_function
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import shutil
import pandas as pd
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import gzip,pickle,sys
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
#from keras import backend as K
from sklearn import metrics

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 88 from PyObject


In [92]:
img_height=60
img_width=60
batch_size= 32
data_dir= "Training_set/Training"
df= pd.read_csv("Training_set/RFMiD_Training_Labels.csv", index_col='ID')
#print (df.head(5))
y=df["Disease_Risk"]
y= np.array(y)
#col_list = df.columns[1:]
print(y[0])

1


In [74]:
col_list = df.columns[:]
print(col_list)
#for index, row in df.iterrows():
    #print (index, " ", row)
#for j, col in enumerate(col_list):
    #print (j, ">>> ", col)

Index(['Disease_Risk', 'DR', 'ARMD', 'MH', 'DN', 'MYA', 'BRVO', 'TSLN', 'ERM',
       'LS', 'MS', 'CSR', 'ODC', 'CRVO', 'TV', 'AH', 'ODP', 'ODE', 'ST',
       'AION', 'PT', 'RT', 'RS', 'CRS', 'EDN', 'RPEC', 'MHL', 'RP', 'CWS',
       'CB', 'ODPM', 'PRH', 'MNF', 'HR', 'CRAO', 'TD', 'CME', 'PTCR', 'CF',
       'VH', 'MCA', 'VS', 'BRAO', 'PLQ', 'HPED', 'CL'],
      dtype='object')


In [72]:
path= "Training_set/Training"
data_dir= os.listdir(path)
#print(data_dir)

In [96]:
img_size= 224

def get_data(path):
    data=[]
    i=0
    data_dir= os.listdir(path)
    for img in data_dir:
        
        img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
        resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
        class_num= y[i]
        i+=1
        data.append([resized_arr,class_num])
        #break
    return np.array(data)

In [97]:
x_data= get_data(path)

<ipython-input-96-a44e9372c218>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


In [98]:
x_data.shape

(1920, 2)

In [75]:

def analyse_classes(dt):
    class_freq = dt.iloc[:, 1:].sum(axis=0).to_dict()
    print('Class Fre',class_freq)
    print("-------------------------------------------------------------------")
    n_diseased = (dt["Disease_Risk"] == 1).sum()
    
    print ('Disease Risk:', n_diseased)
    no_diseased = (dt["Disease_Risk"] == 0).sum()
    print ('No Risk:', no_diseased)
    for c in class_freq:
        print(c, class_freq[c], (class_freq[c] / n_diseased))


In [76]:
analyse_classes(df)


Class Fre {'DR': 376, 'ARMD': 100, 'MH': 317, 'DN': 138, 'MYA': 101, 'BRVO': 73, 'TSLN': 186, 'ERM': 14, 'LS': 47, 'MS': 15, 'CSR': 37, 'ODC': 282, 'CRVO': 28, 'TV': 6, 'AH': 16, 'ODP': 65, 'ODE': 58, 'ST': 5, 'AION': 17, 'PT': 11, 'RT': 14, 'RS': 43, 'CRS': 32, 'EDN': 15, 'RPEC': 22, 'MHL': 11, 'RP': 6, 'CWS': 3, 'CB': 1, 'ODPM': 0, 'PRH': 2, 'MNF': 3, 'HR': 0, 'CRAO': 2, 'TD': 3, 'CME': 4, 'PTCR': 5, 'CF': 3, 'VH': 1, 'MCA': 1, 'VS': 1, 'BRAO': 2, 'PLQ': 1, 'HPED': 1, 'CL': 1}
-------------------------------------------------------------------
Disease Risk: 1519
No Risk: 401
DR 376 0.24753127057274524
ARMD 100 0.06583278472679395
MH 317 0.2086899275839368
DN 138 0.09084924292297564
MYA 101 0.06649111257406189
BRVO 73 0.04805793285055958
TSLN 186 0.12244897959183673
ERM 14 0.009216589861751152
LS 47 0.030941408821593155
MS 15 0.009874917709019092
CSR 37 0.024358130348913758
ODC 282 0.18564845292955892
CRVO 28 0.018433179723502304
TV 6 0.003949967083607637
AH 16 0.01053324555628703
ODP